<a href="https://colab.research.google.com/github/ttezy/Time-Sequence-Analysis/blob/main/VAR_Multivariate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Step 1: Load data**
## **You may need to upload pollution.csv to /content/pollution.csv**

## **Import libs**

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder

import numpy as np

from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.api import VAR

mpl.rcParams['figure.figsize'] = (15, 8)  # modify here to change the size of figures
mpl.rcParams['axes.grid'] = False

## **Load dataset**

In [ ]:
df=pd.read_csv('pollution.csv',header=0, index_col=0)
df

## **Visualize data**

In [ ]:
values = df.values
# specify columns to plot
groups = [0, 1, 2, 3, 5, 6, 7]
i = 1
# plot each column
plt.figure()
for group in groups:
    plt.subplot(len(groups), 1, i)
    plt.plot(values[:, group])
    plt.title(df.columns[group], y=0.5, loc='right')
    i += 1
plt.show()

# **Step 2: Data Property Analysis**

## **Encode wind direction**

In [ ]:
# integer encode direction
encoder = LabelEncoder()
values[:, 4] = encoder.fit_transform(values[:, 4])
# ensure all data is float
values = values.astype('float32')

df['wnd_dir'] = values[:, 4]

df.head()

## **Check stationary or not**

In [5]:
for i in range(len(df.columns)):
  result = adfuller(df[df.columns[i]])

  if result[1] > 0.05 :
    print('{} - Series is not Stationary'.format(df.columns[i]))
  else:
    print('{} - Series is Stationary'.format(df.columns[i]))

pollution - Series is Stationary
dew - Series is Stationary
temp - Series is Stationary
press - Series is Stationary
wnd_dir - Series is Stationary
wnd_spd - Series is Stationary
snow - Series is Stationary
rain - Series is Stationary


## **Choose lag = 8 and show P_values**

In [6]:
max_lags=8  # modify here to choose the lag
y='pollution'

In [7]:
for i in range(len(df.columns)-1):
  results=grangercausalitytests(df[[y,df.columns[i+1]]], max_lags, verbose=False)
  p_values=[round(results[i+1][0]['ssr_ftest'][1],4) for i in range(max_lags)]
  print('Column - {} : P_Values - {}'.format(df.columns[i+1],p_values))

Column - dew : P_Values - [0.0006, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Column - temp : P_Values - [0.0047, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Column - press : P_Values - [0.7517, 0.0933, 0.0001, 0.0, 0.0, 0.0, 0.0, 0.0]
Column - wnd_dir : P_Values - [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Column - wnd_spd : P_Values - [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Column - snow : P_Values - [0.135, 0.3248, 0.2946, 0.4104, 0.419, 0.5307, 0.616, 0.6595]
Column - rain : P_Values - [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## **Choose the input data**

In [8]:
df_input=df[['pollution','dew','temp', 'press', 'wnd_dir', 'wnd_spd', 'snow', 'rain']]  # modify here to choose the variate
df_input

,pollution,dew,temp,press,wnd_dir,wnd_spd,snow,rain
date,,,,,,,,
2010-01-01 00:00:00,0.0,-21,-11.0,1021.0,1.0,1.79,0,0
2010-01-01 01:00:00,0.0,-21,-12.0,1020.0,1.0,4.92,0,0
2010-01-01 02:00:00,0.0,-21,-11.0,1019.0,1.0,6.71,0,0
2010-01-01 03:00:00,0.0,-21,-14.0,1019.0,1.0,9.84,0,0
2010-01-01 04:00:00,0.0,-20,-12.0,1018.0,1.0,12.97,0,0
...,...,...,...,...,...,...,...,...
2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,1.0,231.97,0,0
2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,1.0,237.78,0,0
2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,1.0,242.70,0,0


# **Step 3: Prepare for model**

## **Split training and testing data**

In [9]:
df_train = df_input[:int(0.9*(len(df_input)))]
print('train shape: ', str(df_train.shape))
df_test = df_input[int(0.9*(len(df_input))):]
print('test shape: ', str(df_test.shape))

train shape:  (39441, 8)
test shape:  (4383, 8)


# **Step 4: Train VAR Model**

## **Fit to model and find the best lag number**

In [10]:
model = VAR(df_train, freq="1H")
order = 48 # modify here to choose the order
# for i in range(order):
#     results = model.fit(i+1)
#     print('Order = ', i+1)
#     print('AIC: ', results.aic)
#     print('BIC: ', results.bic)

In [ ]:
model.select_order(order).summary()

## **Choose lag = 25 and fit model again**

In [ ]:
model = VAR(df_train, freq="1H")
results = model.fit(25)

In [ ]:
# print(results.summary())  # show the weight of each variate in lag 25

In [ ]:
lag=results.k_ar
print(lag)

# **Step 5: Implement Data Forecasting**

## **Do a small test**

In [ ]:
pred = np.zeros(0)
for i in range(25, 30):
  pred_next = results.forecast(df_test.values[i-25:i],steps=1)[:, 0]
  pred = np.append(pred, pred_next)
pred

In [ ]:
df_test[25:30]

In [ ]:
plt.plot(df_test['pollution'].values[25:30], label='Ground Truth')
plt.plot(pred, label='Prediction')
plt.title("VAR")
plt.legend()
plt.show()

## **Show the coefficient and their P_Values**

In [ ]:
df_coeff=pd.DataFrame([results.params['pollution'],results.pvalues['pollution']]).T
df_coeff

## **Time series forecasting by VAR**

In [ ]:
pred = np.zeros(25)
print("test shape: ", str(df_test.shape))
for i in range(25, int(len(df_test['pollution']))):
  pred_next = results.forecast(df_test.values[i-25:i],steps=1)[:, 0]
  pred = np.append(pred, pred_next)
print("prediction shape: ", str(pred.shape))

In [ ]:
plt.plot(df_test['pollution'].values[25:200], label='Ground Truth')
plt.plot(pred[25:200], label='Prediction')
plt.title("VAR")
plt.legend()
plt.show()

In [ ]:
plt.plot(df_test['pollution'].values[25:2000], label='Ground Truth')
plt.plot(pred[25:2000], label='Prediction')
plt.title("VAR")
plt.legend()
plt.show()